In [2]:
import numpy as np
import pymysql
import configparser
import json
import time

In [43]:
ip = "139.199.102.41"
config = configparser.RawConfigParser()
config.readfp(open('./settings.cfg'))

dbname = config.get('jx3bla', 'username')
dbpwd = config.get('jx3bla', 'password')
db = pymysql.connect(ip,dbname,dbpwd,"jx3bla",port=3306,charset='utf8')
cursor = db.cursor()

d:\develop\python\python37\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: This method will be removed in future versions.  Use 'parser.read_file()' instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [44]:
edition = "3.5.0"
mapdetail = "25人英雄范阳夜变"

sql = """SELECT * FROM ActorStat WHERE edition>='3.5.0' AND mapdetail='454'"""
cursor.execute(sql)
result = cursor.fetchall()

In [52]:
edition = "3.5.0"
mapdetail = "25人英雄敖龙岛"

sql = """SELECT * FROM ActorStat WHERE edition>='3.5.0' AND mapdetail='428'"""
cursor.execute(sql)
result = cursor.fetchall()

In [53]:
bossDict = {}

for line in result:
    if line[1] not in bossDict:
        bossDict[line[1]] = []
    stat = json.loads(line[6].replace("'", '"'))
    for record in stat["effectiveDPSList"]:
        bossDict[line[1]].append(record)

In [40]:
DPSpos = {"周贽": 2,
          "狼牙精锐": 2,
          "厌夜": 2,
          "迟驻": 5,
          "白某": 6,
          "安小逢": 2}

occName = {"0": (0, 0, 0), 
     "1": "少林",
     "2": "万花",
     "4": "纯阳",
     "5": "七秀",
     "3": "天策",
     "8": "藏剑",
     "9": "丐帮",
     "10": "明教",
     "6": "五毒",
     "7": "唐门",
     "21": "苍云",
     "22": "长歌",
     "23": "霸刀",
     "24": "蓬莱",
     "25": "凌雪",
}

for name in bossDict:
    if name == "狼牙精锐":
        continue
    if name not in DPSpos:
        DPSpos[name] = 2
    allStat = bossDict[name]
    occDict = {}
    num = 0
    for line in allStat:
        if line[1] not in occDict:
            occDict[line[1]] = []
        if line[DPSpos[name]] > 30000:
            occDict[line[1]].append(line[DPSpos[name]])
            num += 1
    table = []
    for occ in occDict:
        data = np.array(occDict[occ])
        res = [occ, occName[occ], data.shape[0], np.mean(data), np.median(data), np.max(data), np.min(data)]
        table.append(res)
    table.sort(key = lambda x: -x[3])
    print("%s的面板DPS统计，样本数：%d"%(name, num))
    for line in table:
        print(line)

驺吾的面板DPS统计，样本数：375
['21', '苍云', 37, 83165.77676950865, 85660.41489361702, 113039.56904761905, 45757.08933717579]
['9', '丐帮', 13, 77598.35202844082, 83904.35207823961, 110676.01699029126, 41663.246411483255]
['25', '凌雪', 23, 77356.95585516, 74513.8275862069, 116516.22478386167, 36290.44009779951]
['8', '藏剑', 23, 76220.62124334567, 70062.26097560975, 105089.38219895288, 49163.668103448275]
['24', '蓬莱', 12, 73901.03593492304, 74278.36495223452, 97737.88349514564, 52969.738505747126]
['7', '唐门', 35, 71801.95629874169, 68903.68318965517, 125822.87277353689, 34145.22964509395]
['22', '长歌', 10, 71791.85860987482, 66827.26532769555, 105144.71726190476, 46694.10051546392]
['3', '天策', 23, 69850.79860706562, 68779.696, 104921.1783919598, 31567.7085106383]
['2', '万花', 24, 66569.57205917034, 64411.97544983819, 99884.43765903308, 41890.15273775216]
['4', '纯阳', 52, 66020.52306660904, 65838.62992424243, 108511.04705882353, 37436.302798982186]
['6', '五毒', 36, 64509.3449798183, 64637.03882095635, 87708.

In [54]:
DPSpos = {"周贽": 2,
          "狼牙精锐": 2,
          "厌夜": 2,
          "迟驻": 5,
          "白某": 6,
          "安小逢": 2}

occName = {"0": (0, 0, 0), 
     "1": "少林",
     "2": "万花",
     "4": "纯阳",
     "5": "七秀",
     "3": "天策",
     "8": "藏剑",
     "9": "丐帮",
     "10": "明教",
     "6": "五毒",
     "7": "唐门",
     "21": "苍云",
     "22": "长歌",
     "23": "霸刀",
     "24": "蓬莱",
     "25": "凌雪",
}

resultDict = {}

for name in bossDict:
    if name == "狼牙精锐":
        continue
    if name not in DPSpos:
        DPSpos[name] = 2
    allStat = bossDict[name]
    occDict = {}
    num = 0
    for line in allStat:
        if line[1] not in occDict:
            occDict[line[1]] = []
        if line[DPSpos[name]] > 30000:
            if name == "安小逢":
                occDict[line[1]].append(line[2:9])
            else:
                occDict[line[1]].append(line[2:])
            num += 1
    table = []
    for occ in occDict:
        if occ in occName:
            data = np.array(occDict[occ], dtype = 'int32')
            res = [occ, occName[occ], data.shape[0]] +  list(np.mean(data, axis = 0))
            table.append(res)
            resultDict[occ] = res[1:]
            
    J = json.dumps(resultDict).replace('"', "'")
    sql = """INSERT INTO DpsStat VALUES ("%s", "%s", "%s")"""%(
        mapdetail, name, J)
    cursor.execute(sql)
    db.commit()
            
    table.sort(key = lambda x: -x[3])
    print("%s的面板DPS统计，样本数：%d"%(name, num))
    print(resultDict)


驺吾的面板DPS统计，样本数：375
{'5': ['七秀', 44, 55268.09090909091], '4': ['纯阳', 52, 66020.01923076923], '10': ['明教', 9, 59267.22222222222], '7': ['唐门', 35, 71801.4], '6': ['五毒', 36, 64508.97222222222], '21': ['苍云', 37, 83165.35135135135], '8': ['藏剑', 23, 76220.13043478261], '9': ['丐帮', 13, 77597.92307692308], '3': ['天策', 23, 69850.26086956522], '23': ['霸刀', 28, 60555.392857142855], '1': ['少林', 6, 60676.833333333336], '2': ['万花', 24, 66569.08333333333], '22': ['长歌', 10, 71791.4], '25': ['凌雪', 23, 77356.52173913043], '24': ['蓬莱', 12, 73900.58333333333]}
藤原武裔的面板DPS统计，样本数：379
{'5': ['七秀', 44, 69344.38636363637], '4': ['纯阳', 51, 78268.39215686274], '10': ['明教', 8, 67611.5], '7': ['唐门', 35, 83482.85714285714], '6': ['五毒', 37, 76830.7027027027], '21': ['苍云', 41, 94056.21951219512], '8': ['藏剑', 23, 88879.52173913043], '9': ['丐帮', 11, 95608.45454545454], '3': ['天策', 25, 77474.6], '23': ['霸刀', 29, 81444.37931034483], '1': ['少林', 7, 65926.28571428571], '2': ['万花', 21, 82813.90476190476], '22': ['长歌', 12, 847

In [50]:
cursor.execute("DROP TABLE IF EXISTS DpsStat")

sql = """CREATE TABLE DpsStat (
         mapdetail VARCHAR(32), 
         boss VARCHAR(32), 
         statistics VARCHAR(16000)
         ) DEFAULT CHARSET utf8mb4"""
cursor.execute(sql)

db.commit()

In [30]:
J = json.dumps(resultDict)

In [28]:
s = json.loads(J)
print(s)

{'22': ['长歌', 101, 77103.32673267326, 15.841584158415841, 112.32673267326733, 74870.80198019803], '21': ['苍云', 324, 82945.78703703704, 12.018518518518519, 77.1574074074074, 81087.79012345678], '8': ['藏剑', 377, 73233.59151193633, 10.456233421750664, 101.62068965517241, 71841.36339522546], '2': ['万花', 332, 69922.1656626506, 15.5, 291.2469879518072, 67951.6295180723], '4': ['纯阳', 445, 65923.30337078651, 15.87191011235955, 229.58876404494382, 63979.02471910112], '6': ['五毒', 413, 69450.7167070218, 16.854721549636803, 328.58837772397095, 67297.50605326876], '5': ['七秀', 534, 63092.65730337079, 18.762172284644194, 511.44756554307116, 60951.050561797754], '7': ['唐门', 483, 71960.15113871636, 17.014492753623188, 152.0248447204969, 69733.85507246378], '3': ['天策', 269, 71562.57620817843, 14.0, 165.4163568773234, 69752.3940520446], '10': ['明教', 189, 70892.16931216931, 12.783068783068783, 140.46560846560845, 69236.69312169313], '1': ['少林', 183, 69959.80327868853, 11.387978142076502, 281.7868852459016